In [420]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
import math
import sys

import datetime

In [213]:
#Users data
usersData = pd.read_csv('users.dat', sep='::', header=None, names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], engine='python')

#Ratings data
ratingsData = pd.read_csv('ratings.dat', sep='::', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp'], engine='python')
ratings_testData = pd.read_csv('ratings_test.dat', sep='::', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp'], engine='python')

#Movies data
moviesData = pd.read_csv('movies.dat', sep='::', header=None, names=['MovieID', 'Title', 'Genres'], engine='python', encoding='latin1')

In [214]:
#Users data
print(usersData.head())

#Ratings data
print(ratingsData.head())
print(ratings_testData.head())

#Movies data
print(moviesData.head())

   UserID Gender  Age  Occupation Zip-code
0       1      F    1          10    48067
1       2      M   56          16    70072
2       3      M   25          15    55117
3       4      M   45           7    02460
4       5      M   25          20    55455
   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291
   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291
   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)         

In [404]:
df_total = ratings_testData.pivot(index = "UserID", columns ="MovieID", values = "Rating")
X_total = np.array(df_total.to_numpy())
#print(X_total)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kf.split(ratings_testData):
    train_set = ratingsData.iloc[train_index]
    test_set = ratingsData.iloc[test_index]
    #print(train_set)
    #print(test_set)
    
    #First, convert data into a numpy array
    df_train = train_set.pivot(index = "UserID", columns ="MovieID", values = "Rating")
    df_test = test_set.pivot(index = "UserID", columns ="MovieID", values = "Rating")
    X_train = np.array(df_train.to_numpy())
    X_test = np.array(df_test.to_numpy())

a = np.empty((X_total.shape)) 
a[:] = np.nan
a_train, a_test = a, a

for i in range(0, X_train.shape[0]):
    for j in range(0, X_train.shape[1]):
        
        a_train[i, j]=X_train[i, j]

for i in range(0, X_test.shape[0]):
    for j in range(0, X_test.shape[1]):
        
        a_test[i, j]=X_test[i, j]


[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
3381
[[ 5. nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [ 5. nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [423]:
totalMAE = 0
totalRMSE = 0

#df_total = ratings_testData.pivot(index = "UserID", columns ="MovieID", values = "Rating")
df_total = ratingsData.pivot(index = "UserID", columns ="MovieID", values = "Rating")
X_total = np.array(df_total.to_numpy())

kf = KFold(n_splits=5, shuffle=True, random_state=42)

#for train_index, test_index in kf.split(ratings_testData):
for train_index, test_index in kf.split(ratingsData):
    now = datetime.datetime.now()
    print ("Current date and time : ", now.strftime("%Y-%m-%d %H:%M:%S"))
    
    train_set = ratingsData.iloc[train_index]
    test_set = ratingsData.iloc[test_index]
    
    #First, convert data into a numpy array
    df_train = train_set.pivot(index = "UserID", columns ="MovieID", values = "Rating")
    df_test = test_set.pivot(index = "UserID", columns ="MovieID", values = "Rating")
    X_train = np.array(df_train.to_numpy())
    X_test = np.array(df_test.to_numpy())
    
    U, M, SE = matrix_factorization(X_train, K=10, num_iter=75, r=0.05, l=0.005)
    
    predicted_X_train = np.dot(U, M.T)
    
    a_predicted_train = np.empty((X_total.shape))
    a_predicted_train[:] = np.nan
    a_test = np.empty((X_total.shape))
    a_test[:] = np.nan

    for i in range(0, predicted_X_train.shape[0]):
        for j in range(0, predicted_X_train.shape[1]):
            a_predicted_train[i, j]=predicted_X_train[i, j]
            
    print(predicted_X_train)

    for i in range(0, X_test.shape[0]):
        for j in range(0, X_test.shape[1]):
            a_test[i, j]=X_test[i, j]
            
    print(X_test)

    diff = a_test-a_predicted_train
    rmse = np.sqrt(np.nanmean(diff**2))
    totalRMSE += rmse
    print("TotalRMSE", totalRMSE)
    
    


# Calculate the average MAE and RMSE over all folds
#averageMAE = totalMAE / 5
averageRMSE = totalRMSE / 5

#print("The Average MAE is", averageMAE)
print("The Average RMSE is", averageRMSE)

Current date and time : 
2023-10-19 00:16:40
Iteration:  0
Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
Iteration:  31
Iteration:  32
Iteration:  33
Iteration:  34
Iteration:  35
Iteration:  36
Iteration:  37
Iteration:  38
Iteration:  39
Iteration:  40
Iteration:  41
Iteration:  42
Iteration:  43
Iteration:  44
Iteration:  45
Iteration:  46
Iteration:  47
Iteration:  48
Iteration:  49
Iteration:  50
Iteration:  51
Iteration:  52
Iteration:  53
Iteration:  54
Iteration:  55
Iteration:  56
Iteration:  57
Iteration:  58
Iteration:  59
Iteration:  60
Iteration:  61
Iteration:  62
Iteration:  63
Itera

In [438]:
def rmse_func(test_matrix, pred_matrix):
    
    # Mask NaN values in the true matrix
    mask = ~np.isnan(test_matrix)
    
    # Calculate squared differences
    diff = test_matrix[mask] - pred_matrix[mask]
    
    # Calculate root mean squared error
    rmse = np.sqrt(np.mean(diff**2))
   
    return rmse

def matrix_factorization(X, K, num_iter, r, l):
    totalRMSE = 0
    """
    Function that performs matrix factorization to obtain the empty (NaN) points in the given matrix

    Arguments:
        X: Full User-Movie rating matrix 
        K: Number of latent features
        num_iter: Number of iterations
        r: Regularization parameter
        l: Learning rate
        

    Returns:
        U: User-feature matrix
        M: Feature-Movie matrix
            np.dot(U, M.T) will then give the predicted Rating Matrix 
        
    """
    
    #We will follow the steps given in chapter 3.1 of the gravity-Tikk.pdf paper:
    I, J = X.shape
    U = np.random.rand(I, K) #U is an I x K matrix with randomly distributed weights 
    M = np.random.rand(J, K) #M is an K x J matrix with randomly distributed weights 
    
    SE = 0
    
    
     
    for step in range(num_iter):
        print("Iteration: ", step)
        for i in range(I):
            for j in range(J):
                if X[i, j] > 0: #i.e. non-NaN
                    eij = X[i, j] - np.dot(U[i, :], M[j, :])
                    #eij denotes the training error on the (i, j)th example
                    
                    for k in range(K):
                        #For all latent features, apply eq.6&7 in gravity-Tikk.pdf
                        U[i, k] = U[i, k] + l * (2 * eij * M[j, k] - r * U[i, k])
                        M[j, k] = M[j, k] + l * (2 * eij * U[i, k] - r * M[j, k])
                    

                    #SE = SE + eij**2 
         
        
        predicted_X = np.dot(U, M.T)
        rmse = rmse_func(X, predicted_X)
        print("RMSE:", rmse)
        
        print(X)
        print(predicted_X)
        
        
        

    
    return U, M, SE

#----------------------------------------------------------------------------------------------------------------
#To test on the array H that is given in the book chapter 9.4:
H = np.array([[5,2,4,4,3],[3,1,2,4,1],[2,np.nan,3,1,4],[2,5,4,3,5],[4,4,5,4,np.nan]]) #let the blank elements be NaN
X = H

#----------------------------------------------------------------------------------------------------------------
#To test on larger array, we can introduce C
# C = np.random.randint(10, 50, size=(100, 100)) #generate values between 10 and 50 because we multiply by 0.1 later

# n_b = 9500 # number of nan we want to add 
# C = C*0.1 # converting the data to float as nan is also of type float
 
# index_C = np.random.choice(C.size, n_b, replace=False) # choosing random indexes to put NaN
# C.ravel()[index_C] = np.nan # adding nan to the data.

# C_new = np.round(C) #To get integer values between 1 and 5
# print(C)
# print(C_new)
# X = C
#----------------------------------------------------------------------------------------------------------------
totalRMSE = 0
U, M, SE = matrix_factorization(X, K=10, num_iter=50, r=0.05, l=0.005)
predicted_X = np.dot(U, M.T)



RMSE = rmse_with_nan(X, predicted_X)
print("The RMSE is", RMSE)

print("Original Rating Matrix:")
print(X)
print("\nPredicted Rating Matrix:")
print(predicted_X)




Iteration:  0
RMSE: 1.4926850655257753
[[ 5.  2.  4.  4.  3.]
 [ 3.  1.  2.  4.  1.]
 [ 2. nan  3.  1.  4.]
 [ 2.  5.  4.  3.  5.]
 [ 4.  4.  5.  4. nan]]
[[2.06004895 2.92392311 2.08274369 3.10067763 3.10410324]
 [2.01421876 2.43813352 2.61495679 2.96911054 2.81320839]
 [1.93441118 2.74769992 2.68867118 3.17417428 3.16972402]
 [2.39423737 2.54655072 2.26482284 2.73725807 3.24471629]
 [2.15827929 2.92536842 2.56094934 2.93400023 3.05157244]]
Iteration:  1
RMSE: 1.3561027099052023
[[ 5.  2.  4.  4.  3.]
 [ 3.  1.  2.  4.  1.]
 [ 2. nan  3.  1.  4.]
 [ 2.  5.  4.  3.  5.]
 [ 4.  4.  5.  4. nan]]
[[2.32405689 3.034599   2.38063955 3.23793019 3.21472564]
 [2.11870951 2.39983823 2.71349268 2.93263371 2.75360041]
 [2.04287347 2.74688741 2.80769884 3.12744475 3.16041482]
 [2.64829442 2.75865929 2.59212835 2.92765379 3.43877036]
 [2.45813429 3.11580525 2.92276283 3.13690654 3.22582947]]
Iteration:  2
RMSE: 1.2479058608334963
[[ 5.  2.  4.  4.  3.]
 [ 3.  1.  2.  4.  1.]
 [ 2. nan  3.  1.  4.]
